# AtmoML Hyperparameter Tuning

In [1]:
import logging
import os
import pathlib
import time

import keras
import keras_tuner
import tensorflow as tf

from usl_models.atmo_ml import dataset, vars, visualizer
from usl_models.atmo_ml.model import AtmoModel

for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

logging.getLogger().setLevel(logging.WARNING)
keras.utils.set_random_seed(812)
visualizer.init_plt()

batch_size = 8
filecache_dir = pathlib.Path("/home/shared/climateiq/filecache")
example_keys = [
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-25"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-26"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-27"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-28"),
    ("PHX_Heat_Test/PHX_summer_2008_25p", "2008-05-25"),
    ("PHX_Heat_Test/PHX_summer_2008_25p", "2008-05-26"),
    ("PHX_Heat_Test/PHX_summer_2008_25p", "2008-05-27"),
    ("PHX_Heat_Test/PHX_summer_2008_25p", "2008-05-28"),
]
timestamp = time.strftime("%Y%m%d-%H%M%S")

ds_config = dataset.Config(output_timesteps=2)
train_ds = dataset.load_dataset_cached(
    filecache_dir,
    example_keys=example_keys,
    config=ds_config,
).batch(batch_size=batch_size)
val_ds = dataset.load_dataset_cached(
    filecache_dir,
    example_keys=example_keys,
    config=ds_config,
    shuffle=False,
).batch(batch_size=batch_size)
# Create a MirroredStrategy to parallelize training on multiple GPUs
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

2025-10-23 04:37:03.090592: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-23 04:37:03.142324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-23 04:37:03.142359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-23 04:37:03.143709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-23 04:37:03.151521: I tensorflow/core/platform/cpu_feature_guar

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


2025-10-23 04:37:06.589046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38364 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [2]:
with strategy.scope():
    tuner = keras_tuner.BayesianOptimization(
        AtmoModel.get_hypermodel(
            input_cnn_kernel_size=[1, 2, 5],
            lstm_kernel_size=[5],
            spatial_activation=["relu"],
            st_activation=["relu"],
            lstm_activation=["relu"],
            output_activation=["tanh"],
        ),
        objective="val_loss",
        max_trials=10,
        project_name=f"logs/htune_project_{timestamp}",
    )
    tuner.search_space_summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [3]:
log_dir = f"logs/htune_{timestamp}"
print(log_dir)
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
tuner.search(train_ds, epochs=100, validation_data=val_ds, callbacks=[tb_callback])
best_model, best_hp = tuner.get_best_models()[0], tuner.get_best_hyperparameters()[0]
best_hp.values

Trial 3 Complete [00h 05m 11s]
val_loss: 0.0029372309800237417

Best val_loss So Far: 0.0029372309800237417
Total elapsed time: 00h 11m 26s


{'input_cnn_kernel_size': 1,
 'lstm_kernel_size': 5,
 'spatial_activation': 'relu',
 'st_activation': 'relu',
 'lstm_activation': 'relu',
 'output_activation': 'tanh'}

In [4]:
with strategy.scope():
    # Re-create the model using the best hyperparameters
    final_params = AtmoModel.Params(**best_hp.values)
    model = AtmoModel(params=final_params)
    tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
    model.fit(train_ds, val_ds, epochs=200, callbacks=[tb_callback], validation_freq=1)
    model.save_model(log_dir + "/model")

Epoch 1/200


2025-10-23 04:51:20.094517: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm_1/conv_lstm/conv_lstm2d_1/while/body/_15/atmo_conv_lstm_1/conv_lstm/conv_lstm2d_1/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


      1/Unknown - 11s 11s/step - loss: 0.2285 - mean_absolute_error: 0.4572 - root_mean_squared_error: 0.4781 - mean_squared_logarithmic_error: 0.1384 - nrmse: 0.5258 - ssim_metric: 0.0086 - psnr_metric: 6.4298 - mse_RH2: 0.1775 - mse_T2: 0.2492 - mse_WSPD_WDIR10: 0.2589

2025-10-23 04:51:25.386779: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6029357418312307068
2025-10-23 04:51:25.386822: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8540019476851868653
2025-10-23 04:51:25.386832: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17858088091765573427
2025-10-23 04:51:25.386840: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1095579078157502711
2025-10-23 04:51:25.386847: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17156103077428152583
2025-10-23 04:51:25.386854: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9088308206256540611
2025-10-23 04:51:25.386861: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

1/1 [==============================] - 14s 14s/step - loss: 0.2285 - mean_absolute_error: 0.4572 - root_mean_squared_error: 0.4781 - mean_squared_logarithmic_error: 0.1384 - nrmse: 0.5258 - ssim_metric: 0.0043 - psnr_metric: 3.2149 - mse_RH2: 0.1775 - mse_T2: 0.2492 - mse_WSPD_WDIR10: 0.2589 - val_loss: 2.8272 - val_mean_absolute_error: 0.8532 - val_root_mean_squared_error: 1.6814 - val_mean_squared_logarithmic_error: 0.3331 - val_nrmse: 1.8494 - val_ssim_metric: 0.0844 - val_psnr_metric: -0.1931 - val_mse_RH2: 6.5061 - val_mse_T2: 1.6032 - val_mse_WSPD_WDIR10: 0.3724
Epoch 2/200


2025-10-23 04:51:28.348185: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1304078566162464939
2025-10-23 04:51:28.348243: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6474579076945936977
2025-10-23 04:51:28.348254: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12927258683125253307
2025-10-23 04:51:28.348262: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10998840067309339905
2025-10-23 04:51:28.348269: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14409533621203174957
2025-10-23 04:51:28.348301: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10463999100196440297
2025-10-23 04:51:28.348317: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

1/1 [==============================] - 3s 3s/step - loss: 2.3157 - mean_absolute_error: 0.7521 - root_mean_squared_error: 1.5218 - mean_squared_logarithmic_error: 0.2807 - nrmse: 1.6738 - ssim_metric: 0.1468 - psnr_metric: 0.4195 - mse_RH2: 5.5561 - mse_T2: 1.1569 - mse_WSPD_WDIR10: 0.2342 - val_loss: 0.2152 - val_mean_absolute_error: 0.4355 - val_root_mean_squared_error: 0.4639 - val_mean_squared_logarithmic_error: 0.1211 - val_nrmse: 0.5103 - val_ssim_metric: 0.1145 - val_psnr_metric: 6.7263 - val_mse_RH2: 0.2075 - val_mse_T2: 0.1901 - val_mse_WSPD_WDIR10: 0.2481
Epoch 3/200
1/1 [==============================] - 3s 3s/step - loss: 0.2139 - mean_absolute_error: 0.4265 - root_mean_squared_error: 0.4625 - mean_squared_logarithmic_error: 0.1183 - nrmse: 0.5087 - ssim_metric: 0.1003 - psnr_metric: 6.7399 - mse_RH2: 0.2012 - mse_T2: 0.2058 - mse_WSPD_WDIR10: 0.2348 - val_loss: 0.3325 - val_mean_absolute_error: 0.4733 - val_root_mean_squared_error: 0.5766 - val_mean_squared_logarithmic_err

INFO:tensorflow:Assets written to: logs/htune_20251023-043706/model/assets


In [ ]:
# Plot results
model = AtmoModel.from_checkpoint(log_dir + "/model")
input_batch, label_batch = next(iter(val_ds))
pred_batch = model.call(input_batch)

for fig in visualizer.plot_batch(
    ds_config,
    input_batch=input_batch,
    label_batch=label_batch,
    pred_batch=pred_batch,
    st_var=vars.Spatiotemporal.TT,
    sto_var=vars.SpatiotemporalOutput.T2,
    max_examples=None,
    dynamic_colorscale=False,  # Set to True to compute from data
    unscale=True,  # Revert normalization to show true values
):
    fig.show()